<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/visualization_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualization Playground
## For 3D Convolutional Neural Network on Group Brain fMRI

This notebook turns fMRI brain images from flat matlab files into 4D tensor objects for CNN training.

To start:
- Mount Google Colab, clone fMRI repository locally, and create path to AWS for saving and loading
- Select desired brain images by subject id, splitting into train, validation, and test sets

Pipeline flow for each batch of images:
- Import desired brain images from AWS paths from data_path_dict
- Drop brain images that are unlabeled
- Mask out the brain, normalize the pixel values, and cast into 4D space
- Aggregate images into tensor-compatible objects for model use
- Upload tensor object dictionary of labels and images to AWS S3
        
  

## Mount Colab in Google Drive and Import Images

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git

# Change directory into cloned repo DL folder
%cd teambrainiac/source/DL

# !ls

fatal: destination path 'teambrainiac' already exists and is not an empty directory.
/content/teambrainiac/source/DL


### Load path_config.py to access AWS credentials

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config (1).py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [ ]:
# Possible Missing Packages
!pip install boto3
!pip install nilearn

# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
import random
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from access_data_dl import *
from process_dl import *

# Pytroch Libraries
import torch


## Import Dictionary of Paths to Flat Matlab Images

In [ ]:
# Open path dictionary file to get subject ids
path = "../data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

In [ ]:
data_path_dict['mask_data']

['svm_data/masks/masks.mat', 'svm_data/masks/roi_masks.mat']

In [ ]:
def labels_mask_binary(data_path_dict, label_type='rt_labels'):
    """
    """
    label_path = data_path_dict['labels'][0]
    label_data_dict = access_load_data(label_path, True)
    labels = label_data_dict[label_type].T[0]
    image_label_mask = np.array([bool(x!=9999) for x in labels])
    image_labels = labels[image_label_mask]

    return image_label_mask, 